# Obtención de datos meteorológicos con la API de la AEMET

Constará de varias funciones para obtener los datos históricos a partir del 22 de junio de la API (en la zona de Cangas de Onís como municipio), y luego las previsiones para los siguientes días (en la zona de montaña de los Picos de Europa). Se construirán algunos dataframe con los que podremos construir un CSV y exportarlo. 

> NOTA: Es necesario proporcionar una API KEY para acceder a la AEMET.

Primero, vamos a agregar la ruta del directorio `keys` del proyecto al PATH de Python.

In [1]:
import sys
import os

In [ ]:
# Ruta absoluta del directorio "keys"
base_dir = os.path.dirname(os.path.abspath(os.getcwd()))
ruta_keys = os.path.join(base_dir, "keys")

# Validación de que el directorio existe
if not os.path.isdir(ruta_keys):
	raise FileNotFoundError(f"El directorio 'keys' no existe en: {ruta_keys}")

# Validación de que el archivo existe
ruta_api_key = os.path.join(ruta_keys, "api_key.py")
if not os.path.isfile(ruta_api_key):
	raise FileNotFoundError(f"El archivo api_key.py no existe en: {ruta_keys}")

# Añadir el directorio al PATH
if ruta_keys not in sys.path:
	sys.path.append(ruta_keys)

In [ ]:
# Paquetes
try:
	from api_key import api_key
except ImportError as e:
	raise ImportError(f"No se pudo importar api_key. Error: {str(e)}")

import requests
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
import time

## Definición de constantes

In [ ]:
querystring = {"api_key": api_key}
headers = {
	'cache-control': "no-cache"
}

ID_MUNICIPIO = '33012'  # Cangas de Onís
ZONA = 'peu1'  # Zona montañosa Picos de Europa
IDEMA = "1178R"  # Sotres
IDEMA2 = "1176"  # Cangas de Onís
IDEMA3 = "1175X"  # Bulnes
hoy = datetime.now(timezone.utc)

fecha_inicio = "2025-06-22T00:00:00UTC"
fecha_fin = "2025-06-29T00:00:00UTC"

fecha_inicio2 = "2025-07-01T00:00:00UTC"
fecha_fin2 = "2025-07-08T00:00:00UTC"

## Obtención de valores climatológicos pasados

Esta función usa el endpoint de la AEMET para obtener los valores climatológicos por días observados en una determinada estación, dentro de un rango de días especificado. Devuelve el dataframe con las columnas de interés.

In [47]:
def consulta_historicos(inicio: str, fin: str, estacion:str=IDEMA) -> pd.DataFrame | None:
	url = f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{inicio}/fechafin/{fin}/estacion/{estacion}"

	# Si la información está ya almacenada, la leemos de un fichero
	if os.path.exists(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{inicio}_{fin}.json")):
		with open(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{inicio}_{fin}.json")) as f:
			prediccion = json.load(f)
	else:
		# Si no, hacemos la petición y guardamos la información devuelta
		response = requests.request("GET", url, headers=headers, params=querystring)

		if response.status_code == 200:

			# Descarga de los datos reales
			datos = response.json()
			url_datos = datos["datos"]
			# print(url_datos)
			response_datos = requests.request("GET", url_datos, headers=headers)

			if response_datos.status_code == 200:
				prediccion = response_datos.json()

				# Almacenar la información devuelta en un fichero
				with open(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{inicio}_{fin}.json"), "w") as f:
					json.dump(prediccion, f)

	if prediccion:
		# Convertir a dataframe y poner la fecha en el índice
		df = pd.DataFrame(prediccion)
		if "fecha" in df.columns:
			df["fecha"] = pd.to_datetime(df["fecha"])

		# Eliminar columnas que no nos interesan
		df_nuevo = df.drop(columns=["indicativo", "nombre", "provincia", "altitud", "horatmin", "horatmax", "tmed", "horaracha", "horaPresMax", "horaPresMin", "horaHrMax", "horaHrMin", "presMax", "presMin", "dir", "racha"])
		
		return df_nuevo

In [48]:
# Cambio en la configuración de pandas para mostrar el dataframe
pd.set_option('display.max_columns', None)      # Muestra todas las columnas
pd.set_option('display.width', None)            # Usa el ancho completo del terminal o Jupyter
pd.set_option('display.expand_frame_repr', False)  # Evita que pandas divida líneas

df_pasado = consulta_historicos(fecha_inicio, fecha_fin)
time.sleep(2)
df_pasado2 = consulta_historicos(fecha_inicio2, fecha_fin2)

# Concatenar ambos dataframes
df_pasado_completo = pd.concat([df_pasado, df_pasado2], ignore_index=True)
print(df_pasado_completo)

        fecha  prec  tmin  tmax velmedia hrMedia hrMax hrMin
0  2025-06-22   0,0  11,9  20,4      1,4      99   100    61
1  2025-06-23   2,0  11,7  25,0      2,5      64   100    40
2  2025-06-24  23,8  13,7  26,6      5,6      54    94    34
3  2025-06-25   2,0  11,5  19,9      3,6      86   100    50
4  2025-06-26   0,0  11,4  18,2      1,4      95   100    80
5  2025-06-27   0,0  14,8  23,5      2,5      58    88    51
6  2025-06-28   0,0  16,7  26,7      2,8      56    90    46
7  2025-06-29   6,0  14,6  28,9      1,4      43    85    28
8  2025-07-01   0,4  14,4  29,1      2,2      61   100    30
9  2025-07-02   5,8  11,4  16,0      3,3     100   100    94
10 2025-07-03  30,6  11,0  17,0      3,6     100   100    70
11 2025-07-04   0,0  11,3  22,8      4,7      75   100    53
12 2025-07-05   0,0  12,3  19,8      3,3      88   100    76
13 2025-07-06   1,6  10,9  18,5      1,4      90   100    70
14 2025-07-07   0,0   7,9  15,0      1,9      96   100    86
15 2025-07-08   0,0   6,

Teniendo esto, debemos transformar las columnas que podamos a enteros.

In [56]:
columnas_convertir_float = ["prec", "tmin", "tmax", "velmedia", "hrMedia", "hrMax", "hrMin"]

# Cambiar las comas decimales por puntos
df_pasado_completo[columnas_convertir_float] = df_pasado_completo[columnas_convertir_float].replace(",", ".", regex=True)

# Hacer las transformaciones de tipos
df_pasado_completo[columnas_convertir_float] = df_pasado_completo[columnas_convertir_float].astype(float)

Una vez tenemos estos datos, vamos a completarlos con la información obtenida en la observación directa y en las previsiones de esos días. Esto tendrá que hacerse manualmente. 
 
> NOTA: Si se quiere usar este programa para otros fines y se buscan otros datos u otras fechas, habrá que rellenar manualmente estas columnas o dejarlas vacías con comillas. La AEMET no almacena un histórico con estos datos, por lo que si el usuario no los ha guardado, no hay manera de acceder a ellos.

In [57]:
if df_pasado_completo is not None:
	# low/high/mid
	df_pasado_completo["altonubes"] = ["low", "", "mid", "", "low", "mid", "mid", "mid", "low", "low", "low", "low", "low", "low", "low", "low"]      

	# abundante/escasa/media
	df_pasado_completo["nubosidad"] = ["abundante", "","escasa", "abundante", "escasa", "escasa", "media", "escasa", "abundante", "abundante", "abundante", "abundante", "abundante", "abundante", "abundante", "escasa"] 

	# chubascos/posible/no
	df_pasado_completo["lluvia"] = ["posible", "", "chubascos", "chubascos", "no", "no", "no", "posible", "posible", "posible", "posible", "posible", "posible", "posible", "no", "no"]

	# flojo/moderado/fuerte
	df_pasado_completo["viento"] = ["moderado", "", "fuerte", "moderado", "flojo", "flojo", "flojo", "moderado", "moderado", "moderado", "moderado", "moderado", "flojo", "moderado", "moderado", "flojo"]

	# SI/NO  (en base a las imágenes finales)
	df_pasado_completo["SUBIR"] = ["NO", "NO", "SI", "SI", "NO", "SI", "SI", "SI", "NO", "NO", "NO", "NO", "NO", "NO", "NO", "SI"]  

	print(df_pasado_completo)

                fecha  prec  tmin  tmax  velmedia  hrMedia  hrMax  hrMin altonubes  nubosidad     lluvia    viento SUBIR
fecha                                                                                                                   
2025-06-22 2025-06-22   0.0  11.9  20.4       1.4     99.0  100.0   61.0       low  abundante    posible  moderado    NO
2025-06-23 2025-06-23   2.0  11.7  25.0       2.5     64.0  100.0   40.0                                              NO
2025-06-24 2025-06-24  23.8  13.7  26.6       5.6     54.0   94.0   34.0       mid     escasa  chubascos    fuerte    SI
2025-06-25 2025-06-25   2.0  11.5  19.9       3.6     86.0  100.0   50.0            abundante  chubascos  moderado    SI
2025-06-26 2025-06-26   0.0  11.4  18.2       1.4     95.0  100.0   80.0       low     escasa         no     flojo    NO
2025-06-27 2025-06-27   0.0  14.8  23.5       2.5     58.0   88.0   51.0       mid     escasa         no     flojo    SI
2025-06-28 2025-06-28   0.0  16.

Una vez tenemos el dataframe completo, vamos a establecer como índice la fecha.

In [58]:
df_pasado_completo.set_index(df_pasado_completo.fecha, inplace=True)
df_pasado_final = df_pasado_completo.drop("fecha", axis=1)
df_pasado_final

,prec,tmin,tmax,velmedia,hrMedia,hrMax,hrMin,altonubes,nubosidad,lluvia,viento,SUBIR
fecha,,,,,,,,,,,,
2025-06-22,0.0,11.9,20.4,1.4,99.0,100.0,61.0,low,abundante,posible,moderado,NO
2025-06-23,2.0,11.7,25.0,2.5,64.0,100.0,40.0,,,,,NO
2025-06-24,23.8,13.7,26.6,5.6,54.0,94.0,34.0,mid,escasa,chubascos,fuerte,SI
2025-06-25,2.0,11.5,19.9,3.6,86.0,100.0,50.0,,abundante,chubascos,moderado,SI
2025-06-26,0.0,11.4,18.2,1.4,95.0,100.0,80.0,low,escasa,no,flojo,NO
2025-06-27,0.0,14.8,23.5,2.5,58.0,88.0,51.0,mid,escasa,no,flojo,SI
2025-06-28,0.0,16.7,26.7,2.8,56.0,90.0,46.0,mid,media,no,flojo,SI
2025-06-29,6.0,14.6,28.9,1.4,43.0,85.0,28.0,mid,escasa,posible,moderado,SI
2025-07-01,0.4,14.4,29.1,2.2,61.0,100.0,30.0,low,abundante,posible,moderado,NO


Por último, podemos guardar el dataframe construido en un CSV para procesarlo más tarde:

In [59]:
df_pasado_final.to_csv(os.path.join(base_dir, "data", "processed", "data.csv"), sep=";", encoding="utf-8")

## Obtención de previsiones en zona de montaña

Esta función imprime las previsiones de los próximos 4 días (desde hoy hasta dentro de 3 días). Como se dan de forma textual, esto servirá para crear manualmente el dataframe.

Como la API de la AEMET devuelve la previsión en forma de texto, vamos a intentar imputar el valor de las columnas que nos interesan (`altonubes`, `nubosidad`, `lluvia`, `viento`) procesando la salida textual con un LLM ejecutado en local con Ollama.

> NOTA: Para ejecutar esto, se necesita tener en ejecución el modelo `gemma:2b` de Ollama en el sistema. También se puede emplear otro, indicándolo como parámetro.

### Funciones para procesar un texto con un LLM e imputar valores

Comenzamos tratando de procesar el texto sobre las nubes.

In [28]:
def consulta_ollama_nubes(texto, modelo='gemma:2b'):
	url = "http://localhost:11434/api/generate"
	headers = {"Content-Type": "application/json"}

	prompt = f"""Extrae las siguientes dos variables meteorológicas a partir del siguiente informe de predicción:
- 'altonubes': puede ser 'low', 'medium' o 'high'
- 'nubosidad': puede ser 'abundante', 'media' o 'escasa'

Texto:
\"\"\"{texto}\"\"\"

Respuesta en formato JSON estrictamente, sin incluir nada más. Debe tener **una sola clave para cada variable**, por ejemplo:
\"{{"altonubes": "alta", "nubosidad": "moderada"}}\".
Asegúrate de respetar siempre tanto los nombres de los campos como los posibles valores.
"""

	data = {
		"model": modelo,
		"prompt": prompt,
		"stream": False,  # para que la respuesta venga entera, no en streaming
		"temperature": 0.0   # para hacerlo determinista
	}

	response = requests.post(url, headers=headers, data=json.dumps(data))
	if response.status_code == 200:
		return response.json()["response"]
	else:
		raise Exception(f"Error {response.status_code}: {response.text}")


Realizamos una prueba:

In [29]:
print(consulta_ollama_nubes("Intervalos de nubes bajas a primeras horas, y ocasionalmente a últimas, que serán más frecuentes en la vertiente cantábrica, y que localmente podrían reducir la visibilidad. Predominio de cielos poco nubosos o despejados el resto del tiempo"))

{"altonubes": "low", "nubosidad": "moderada"}


Viendo que esta función devuelve el resultado esperado, vamos a adaptarla para imputar los valores de `lluvia` y `viento`:

In [26]:
def consulta_ollama_lluvia(texto, modelo='gemma:2b'):
	url = "http://localhost:11434/api/generate"
	headers = {"Content-Type": "application/json"}

	prompt = f"""Extrae la siguiente variable meteorológicas a partir del siguiente informe de predicción:
- 'lluvia': puede ser 'chubascos', 'posible' o 'no'

Texto:
\"\"\"{texto}\"\"\"

Respuesta en formato JSON. La variable 'lluvia' debe tomar uno de los 3 valores que te especifico. La clave del JSON debe ser la palabra 'lluvia' siempre, y el valor debe ser uno de los tres que pongo, sin excepción.
"""

	data = {
		"model": modelo,
		"prompt": prompt,
		"stream": False,  # para que la respuesta venga entera, no en streaming
		"temperature": 0.0   # para hacerlo determinista
	}

	response = requests.post(url, headers=headers, data=json.dumps(data))
	if response.status_code == 200:
		return response.json()["response"]
	else:
		raise Exception(f"Error {response.status_code}: {response.text}")
	

def consulta_ollama_viento(texto, modelo='gemma:2b'):
	url = "http://localhost:11434/api/generate"
	headers = {"Content-Type": "application/json"}

	prompt = f"""Extrae la siguiente variable meteorológicas a partir del siguiente informe de predicción:
- 'viento': puede ser 'fuerte', 'moderado' o 'flojo'

Texto:
\"\"\"{texto}\"\"\"

Respuesta en formato JSON. La variable 'viento' debe tomar uno de los 3 valores que te especifico. La clave del JSON debe ser la palabra 'viento' siempre, y el valor debe ser uno de los tres que pongo, sin excepción.
"""

	data = {
		"model": modelo,
		"prompt": prompt,
		"stream": False,  # para que la respuesta venga entera, no en streaming
		"temperature": 0.0   # para hacerlo determinista
	}

	response = requests.post(url, headers=headers, data=json.dumps(data))
	if response.status_code == 200:
		return response.json()["response"]
	else:
		raise Exception(f"Error {response.status_code}: {response.text}")

Pruebas:

In [27]:
print(consulta_ollama_lluvia("Escasa probabilidad de alguna lluvia débil en la vertiente cantábrica de madrugada"))
print(consulta_ollama_viento("De componente norte, generalmente flojo, girando a oeste en cotas altas y a este en el resto"))

{"lluvia": "no"}
{"viento": "flojo"}


Una vez tenemos ya las funciones para procesar la descripción textual, podemos ya rellenar un JSON propio del estilo de los demás. Este JSON se convertirá luego en otro CSV, que contendrá los datos para los cuales la variable `SUBIR` no toma valor, y se tendrá que predecir con el modelo de ML que construiremos. 

In [34]:
def consulta_mountain(zona=ZONA):
	url = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/montaña/pasada/area/{zona}/dia/"
	list_dict_respuesta = []

	for i in range(4):

		# Inicialización de un diccionario
		list_dict_respuesta.append(dict())

		# Consulta a la AEMET	
		response = requests.request("GET", url+str(i), headers=headers, params=querystring)

		if response.status_code == 200:

			# Descarga de los datos reales
			datos = response.json()
			url_datos = datos["datos"]
			
			response_datos = requests.request("GET", url_datos, headers=headers)

			if response_datos.status_code == 200:
				prediccion = response_datos.json()

				# En base a la respuesta dada, rellenamos un JSON propio
				for diccionario in prediccion[0]["seccion"][0]["apartado"]:
					if diccionario["cabecera"] == "Estado del cielo":
						respuesta_nubes = consulta_ollama_nubes(diccionario["texto"])
						if respuesta_nubes.strip():
							try:
								datos_nubes = json.loads(respuesta_nubes)
								list_dict_respuesta[i]["altonubes"] = datos_nubes["altonubes"]
								list_dict_respuesta[i]["nubosidad"] = datos_nubes["nubosidad"]
							except json.JSONDecodeError as e:
								print(f"❌ Error decodificando JSON:\n{respuesta_nubes}")
								print(e)
						else:
							print(f"❌ Respuesta vacía de Ollama para texto:\n{diccionario['texto']}")

					
					if diccionario["cabecera"] == "Precipitaciones":
						respuesta_prec = consulta_ollama_lluvia(diccionario["texto"])
						if respuesta_prec.strip():
							try:
								datos_lluvia = json.loads(respuesta_prec)
								list_dict_respuesta[i]["lluvia"] = datos_lluvia["lluvia"]
							except json.JSONDecodeError as e:
								print(f"❌ Error decodificando JSON:\n{respuesta_prec}")
								print(e)
					

					if diccionario["cabecera"] == "Viento":
						respuesta_viento = consulta_ollama_viento(diccionario["texto"])
						if respuesta_viento.strip():
							try:
								datos_viento = json.loads(respuesta_viento)
								list_dict_respuesta[i]["viento"] = datos_viento["viento"]
							except json.JSONDecodeError as e:
								print(f"❌ Error decodificando JSON:\n{respuesta_prec}")
								print(e)
				
				# Almacenar la fecha
				today = datetime.today()
				date = today + timedelta(days=i)
				date_formatted = date.strftime('%Y-%m-%d')
				list_dict_respuesta[i]["fecha"] = date_formatted

	return list_dict_respuesta
			

Ahora, ejecutamos la función para crear el JSON (lista de diccionarios), y lo convertimos a dataframe de pandas. 

In [35]:
json_salida = consulta_mountain()
# Convertir a dataframe y poner la fecha en el índice
df_mountain = pd.DataFrame(json_salida)
if "fecha" in df_mountain.columns:
	df_mountain["fecha"] = pd.to_datetime(df_mountain["fecha"])
	df_mountain.set_index(df_mountain.fecha, inplace=True)
	df_mountain = df_mountain.drop("fecha", axis=1)
df_mountain

❌ Error decodificando JSON:
{"lluvia": "no"}
Extra data: line 2 column 1 (char 20)


,altonubes,nubosidad,lluvia,viento
fecha,,,,
2025-07-25,low,media,no,flojo
2025-07-26,low,moderada,no,flojo
2025-07-27,alta,moderada,no,flojo
2025-07-28,alta,moderada,no,NaN


Una vez tenemos esto, lo complementamos con la previsión en los próximos 4 días en el municipio de Cangas de Onís, de forma que podamos obtener el resto de datos numéricos y añadirlos al dataframe como valores de columnas. 

Para hacerlo, vamos a crear otro dataframe de la misma manera que el anterior, pero con las nuevas columnas, y luego concatenaremos los dos. 

In [53]:
def add_prevision_municipio(municipio=ID_MUNICIPIO):
	'''
	Función para consultar las previsiones meteorológicas en los próximos 4 días en la zona de un municipio indicado
	mediante el parámetro, y añadirla a una estructura tipo JSON.
	'''
	
	list_dict_prevision = []

	url = f"https://opendata.aemet.es/opendata/api/prediccion/especifica/municipio/diaria/{municipio}"

	# Si la información está ya almacenada, la leemos de un fichero
	if os.path.exists(os.path.join(base_dir, "data", "raw", f"datos_aemet_prevision_{datetime.today().strftime('%Y-%m-%d')}.json")):
		with open(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{datetime.today().strftime('%Y-%m-%d')}.json")) as f:
			prediccion = json.load(f)
	else:
		# Si no, hacemos la petición y guardamos la información devuelta
		response = requests.request("GET", url, headers=headers, params=querystring)

		if response.status_code == 200:

			# Descarga de los datos reales
			datos = response.json()
			url_datos = datos["datos"]
			response_datos = requests.request("GET", url_datos, headers=headers)

			if response_datos.status_code == 200:
				prediccion = response_datos.json()

				# Almacenar la información devuelta en un fichero
				with open(os.path.join(base_dir, "data", "raw", f"datos_aemet_historicos_{datetime.today().strftime('%Y-%m-%d')}.json"), "w") as f:
					json.dump(prediccion, f)

	
	if prediccion:
		# Procesar el JSON y obtener un diccionario como el que teníamos
		for dia in range(4):
			list_dict_prevision.append(dict())
			prevision_completa_dia = prediccion[0]["prediccion"]["dia"][dia]
			list_dict_prevision[dia]["fecha"] = prevision_completa_dia["fecha"][0:10]
			list_dict_prevision[dia]["prec"] = prevision_completa_dia["probPrecipitacion"][0]["value"]
			list_dict_prevision[dia]["tmin"] = prevision_completa_dia["temperatura"]["minima"]
			list_dict_prevision[dia]["tmax"] = prevision_completa_dia["temperatura"]["maxima"]
			list_dict_prevision[dia]["velmedia"] = prevision_completa_dia["viento"][0]["velocidad"]
			list_dict_prevision[dia]["hrMax"] = prevision_completa_dia["humedadRelativa"]["maxima"]
			list_dict_prevision[dia]["hrMin"] = prevision_completa_dia["humedadRelativa"]["minima"]
			list_dict_prevision[dia]["hrMedia"] = round(sum(map(lambda json: json["value"], prevision_completa_dia["humedadRelativa"]["dato"]))/4, 1)
	
		return list_dict_prevision

Una vez tenemos la función, transformamos el resultado que devuelve a otro pandas como el anterior:

In [54]:
json_salida_prevision = add_prevision_municipio()

# Convertir a dataframe y poner la fecha en el índice
df_prev = pd.DataFrame(json_salida_prevision)
if "fecha" in df_prev.columns:
	df_prev["fecha"] = pd.to_datetime(df_prev["fecha"])
	df_prev.set_index(df_prev.fecha, inplace=True)
	df_prev = df_prev.drop("fecha", axis=1)
df_prev

,prec,tmin,tmax,velmedia,hrMax,hrMin,hrMedia
fecha,,,,,,,
2025-07-25,0,14,23,0,90,60,77.5
2025-07-26,0,11,25,10,100,60,80.0
2025-07-27,85,12,26,10,100,50,0.0
2025-07-28,90,15,25,10,90,55,0.0


Por último, tendríamos que concatenar ambos dataframes de las previsiones de los próximos 4 días. Para ello, hacemos un outer JOIN al estilo de SQL.

In [55]:
df_previsiones_completo = df_prev.join(df_mountain, how="outer")
df_previsiones_completo

,prec,tmin,tmax,velmedia,hrMax,hrMin,hrMedia,altonubes,nubosidad,lluvia,viento
fecha,,,,,,,,,,,
2025-07-25,0,14,23,0,90,60,77.5,low,media,no,flojo
2025-07-26,0,11,25,10,100,60,80.0,low,moderada,no,flojo
2025-07-27,85,12,26,10,100,50,0.0,alta,moderada,no,flojo
2025-07-28,90,15,25,10,90,55,0.0,alta,moderada,no,NaN


Realizamos las mismas conversiones de tipos que antes:

In [60]:
columnas_convertir_float = ["hrMedia", "hrMax", "hrMin", "prec", "tmin", "tmax", "velmedia"]

# Cambiar las comas decimales por puntos
df_previsiones_completo[columnas_convertir_float] = df_previsiones_completo[columnas_convertir_float].replace(",", ".", regex=True)

# Hacer las transformaciones de tipos
df_previsiones_completo[columnas_convertir_float] = df_previsiones_completo[columnas_convertir_float].astype(float)

Lo almacenamos en un CSV correspondiente a los datos cuya etiqueta (SUBIR) querremos predecir. Serán nuestros casos con los que probaremos el modelo de Machine Learning.

In [61]:
df_previsiones_completo.to_csv(os.path.join(base_dir, "data", "processed", "data_sin_etiquetar.csv"), sep=";", encoding="utf-8")